# Setup and Imports

Additional requirements:
- hyperopt 0.2.4
- lightgbm 2.3.1

In [ ]:
import sys
sys.path.insert(0,'../..')  # add project root to PATH

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# supress tensorflow depreciation warnings
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [ ]:
# imports
from src.model import TinyYoloV3, train_test_split
from src.preprocessing import JSONUtil
from pathlib import Path
import numpy as np
from hyperopt import Trials, STATUS_OK, tpe, fmin, hp, atpe

In [ ]:
data_path = '../../data/'
annotation_data = JSONUtil.read(Path(data_path, "Master.json"))
weights_path = '../../models/pre_trained_weights/tiny-yoloV3.h5'

# Model Training

### Set the Random Seeds
For deterministic execution.

In [ ]:
# set seeds
tf.random.set_random_seed(4321)
np.random.seed(1459)
split_seed = 2345

### Load and Prepare the Data
Split the data into a test, validation and training set.

In [ ]:
# Train-Test split
test_split = 0.1
train_val_data, test_data = train_test_split(annotation_data, test_split, split_seed)

# Train-Validation split
validation_split = 0.2
training_data, validation_data = train_test_split(train_val_data, validation_split, split_seed)

## Transfer Learning: Freezing followed by fine-tuning the whole model

### Define the Parameters

In [ ]:
batch_size = 16
lr_freeze = 10**-2
lr_fine_tuning = 10**-6
epochs_freeze = 10
epochs_fine_tuning = 10

# Output paths
out_path_freeze = "model_1208.h5"
out_path_fine_tuning = "model_1208_fine_tune.h5"
out_path_checkpoints ="weights.best.hdf5"

### Setup the Model

In [ ]:
space = {
            'rotation_probability': hp.uniform('rotation_probability', 0, 1),
            'jittering_probability': hp.uniform('jittering_probability', 0, 1),
            'hue': hp.uniform('hue', 0.1, 1),
            'sat': hp.uniform('sat', 0.1, 1.5), 
            'val': hp.uniform('val', 0.1, 1.5), 
        }

In [ ]:
def opt_fun(params):
    import keras.backend as K  
    from keras.callbacks import EarlyStopping, ReduceLROnPlateau,ModelCheckpoint
    from src.model import TinyYoloV3
    
    # set seeds
    tf.random.set_random_seed(4321)
    np.random.seed(1459)
    split_seed = 2345
    
    # fix these parameters for faster learning of jittering paramters
    epochs_freeze = 15
    batch_size = 16
    
    jittering_params = {
        "rotation_probability": params["rotation_probability"],
        "jittering_probability": params["jittering_probability"],
        "jittering_range": {
            "hue": params["hue"],
            "sat": params["sat"],
            "val": params["val"]
        }
    }

    # Set Keras to learning-mode --> fix constantly adapting batch-normalizations
    K.set_learning_phase(1)

    # Setup Model
    model = TinyYoloV3(path = weights_path, pre_trained_weights = True)
    model.replace_output_layers()
    model.training_mode()
    
    # Define the Callbacks
    early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3, restore_best_weights = True)
    checkpoint = ModelCheckpoint(out_path_checkpoints, monitor='val_loss', verbose=0, save_best_only=True)
    callbacks = [early_stopping, checkpoint]
    
    
    # freeze selected layers
    model.freeze_all_but_output()

    # start training (Transfer Learning)
    history_freeze = model.train(training_data, validation_data, data_path, lr_freeze, batch_size, 
                                 epochs_freeze, out_path=out_path_freeze, callbacks = callbacks, 
                                 jittering_params = jittering_params, verbose=0)
    
    # obtain val-error
    score = model.evaluate(validation_data, data_path)
    
    # display params
    print(params, score)
    
    return {'loss': score, 'status': STATUS_OK, 'model': model, 'params': params}

In [ ]:
# rounds
max_evals = 15

### optimization
trials = Trials()
best = fmin(fn = opt_fun, space = space, algo= atpe.suggest, max_evals= max_evals, trials=trials, rstate= np.random.RandomState(2222))
print('best: ')
### Extract params
print(trials.results[np.argmin([r['loss'] for r in trials.results])]['params'])
# Extract Model
model =trials.results[np.argmin([r['loss'] for r in trials.results])]['model']

## Evaluate the Model: Obtain the Train and Test Error

In [ ]:
# obtain train-error
model.evaluate(train_val_data, data_path)

In [ ]:
# obtain test-error
model.evaluate(test_data, data_path)